In [23]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import BatchNorm1d, Dropout, Linear, ReLU, Sequential
import torch_geometric.transforms as T

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_add_pool

path = osp.join('data', 'TU')
# dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES').shuffle()
dataset = TUDataset(path, name='MUTAG').shuffle()
print(dataset[0])

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256)


class Net(torch.nn.Module):
    def __init__(self, in_channels, dim, out_channels):
        super().__init__()

        self.conv1 = GINConv(
            Sequential(Linear(in_channels, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv3 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv4 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv5 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.lin1 = Linear(dim, dim)
        self.lin2 = Linear(dim, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
        x = self.conv5(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, 32, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        total_correct += int((out.argmax(-1) == data.y).sum())
    return total_correct / len(loader.dataset)


for epoch in range(1, 101):
    loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f} '
          f'Test Acc: {test_acc:.4f}')

Data(edge_index=[2, 26], x=[13, 7], edge_attr=[26, 4], y=[1])
Epoch: 001, Loss: 0.8115, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 002, Loss: 0.6837, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 003, Loss: 0.5235, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 004, Loss: 0.4709, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 005, Loss: 0.4437, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 006, Loss: 0.3764, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 007, Loss: 0.3372, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 008, Loss: 0.3499, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 009, Loss: 0.3238, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 010, Loss: 0.3023, Train Acc: 0.6647 Test Acc: 0.7222
Epoch: 011, Loss: 0.2744, Train Acc: 0.6647 Test Acc: 0.7222
Epoch: 012, Loss: 0.2644, Train Acc: 0.6706 Test Acc: 0.7222
Epoch: 013, Loss: 0.2428, Train Acc: 0.6824 Test Acc: 0.7222
Epoch: 014, Loss: 0.2369, Train Acc: 0.6882 Test Acc: 0.7222
Epoch: 015, Loss: 0.2378, Train Acc: 0.6824 Test Acc: 0.7222
Epoch: 016, Loss: 0.215

In [ ]:
Data(x=[16, 7], edge_attr=[36, 4], y=[1], adj_t=[16, 16, nnz=36])